# Motivation
I want to check out Pinecone, which is a vector database that I can use for semantic search. This notebook will contain a couple of my experiments with Pinecone. 

# Setup
The cells below will help set up the rest of the notebook. 

I'm going to start by changing my working directory to the repo root. 

In [1]:
%cd ..

d:\data\programming\neural-needle-drop


Next, I'll import different modules. 

In [59]:
# Import statements
import pandas as pd
import pinecone
import os
import mysql.connector
import traceback
import numpy as np
import itertools
import math
from tqdm import tqdm
from time import sleep
from time import time

After that, I'm going to set up a connection to my MySQL database. This will help me load in the relevant data. 

In [9]:
# Set up the connection to the MySQL server
cnx = mysql.connector.connect(
    user='root', password=os.getenv("MYSQL_PASSWORD"), 
    host='localhost', database='neural-needle-drop')

# Create a cursor 
cursor = cnx.cursor()

Finally, we're going to set up the connection to the Pinecone API. 

In [4]:
# Initialize the Pinecone API connection
pinecone.init(api_key=os.getenv("PINECONE_API_KEY"))

# Setting up the index 
pinecone_index = pinecone.Index("neural-needledrop-prototype")

# Methods
The cells below will define a couple of methods that'll be important for this notebook.

In [25]:
def query_to_df(query, print_error=False):
    '''Query the active MySQL database and return results in a DataFrame'''

    # Try to return the results as a DataFrame
    try:
        # Execute the query
        cursor.execute(query)

        # Fetch the results 
        res = cursor.fetchall()

        # Return a DataFrame
        return pd.DataFrame(res, columns=[i[0] for i in cursor.description])

    # If we run into an Exception, return None
    except Exception as e:
        if (print_error):
            print(f"Ran into the following error:\n{e}\nStack trace:")
            print(traceback.format_exc())
        return None

def chunks(iterable, batch_size=100):
    """A helper function to break an iterable into chunks of size batch_size."""
    it = iter(iterable)
    chunk = tuple(itertools.islice(it, batch_size))
    while chunk:
        yield chunk
        chunk = tuple(itertools.islice(it, batch_size))

# Uploading Data
In order to start working with the Pinecone index, I wanted to upload some data. I'll start with some of the whole-video embeddings. In order to upload those, I'll need to load them in from the SQL server. 

In [48]:
# This query will grab the entire embeddings table
embeddings_df_query = """SELECT * FROM embeddings"""

# Execute the above query 
embeddings_df = query_to_df(embeddings_df_query, print_error=True)

# Now, convert the embedding column from binary data to a list  
embeddings_df["embedding"] = embeddings_df["embedding"].apply(lambda x: np.frombuffer(x).tolist())

# Create the whole_video_embeddings_df by filtering the embeddings_df
whole_video_embeddings_df = embeddings_df.query("embedding_type=='whole_video'").copy()

I'm also going to load in all of the video details, just for effiency's sake. 

In [ ]:
# Load in all of the data from the video_details table
tnd_data_df = query_to_df(
    """SELECT * FROM video_details""", 
    print_error=True)

With this data loaded, I can start uploading information to Pinecone. 

In [51]:
# We're going to break this DataFrame into chunks of 100
chunk_size = 100
chunk_amt = int(math.ceil(len(whole_video_embeddings_df)/chunk_size))
for cur_chunk_num in tqdm(list(range(chunk_amt))):

    # Subset the DataFrame so that we're only looking at <= 100 videos
    df_chunk = whole_video_embeddings_df[cur_chunk_num*chunk_size:(cur_chunk_num+1)*chunk_size].copy()

    # Format the data from this df_chunk into something you can send to Pinecone
    pinecone_upsert_list = [(row.id, row.embedding, 
    {"embedding_type": row.embedding_type,
     "start_segment": row.start_segment,
     "end_segment": row.end_segment}) for row in df_chunk.itertuples()]
    
    # Upload this chunk to Pinecone
    pinecone_index.upsert(vectors=pinecone_upsert_list, namespace="video_embeddings")

    # Sleep a little bit to avoid overwhelming Pinecone
    sleep(1.5)

100%|██████████| 39/39 [02:19<00:00,  3.58s/it]


# Querying Data
Next, I want to try and query some data using Pinecone. They've got [some documentation on querying here](https://docs.pinecone.io/docs/query-data) - the cells below will try and replicate that.  

In [78]:
# Extract the vector we want to search
to_search_video_id = "QaMpiKZh1fc"
to_search_embedding = whole_video_embeddings_df.query(
    f"id=='{to_search_video_id}'").iloc[0].embedding

# Querying Pinecone 
start_time = time()
pinecone_query_results = pinecone_index.query(
    vector=to_search_embedding,
    top_k=5,
    namespace="video_embeddings")

# Print some information about how long the query took
print(f"That query took {time()-start_time:.2f} seconds.")

That query took 0.11 seconds.


Pretty fast, huh? Let's join the IDs together with their titles to figure out which videos are the most similar:

In [79]:
for match_dict in pinecone_query_results['matches']:
    cur_match_video_details = tnd_data_df.query(f"id=='{match_dict.id}'").iloc[0]
    print(f"{cur_match_video_details.title}\nSIMILARITY: {match_dict.score}\n")

Radiohead - Kid A ALBUM REVIEW
SIMILARITY: 1.0

Radiohead - A Moon Shaped Pool ALBUM REVIEW
SIMILARITY: 0.897760153

Radiohead: Worst To Best
SIMILARITY: 0.897391081

Radiohead - Burn the Witch TRACK REVIEW
SIMILARITY: 0.893643439

Talking Heads - Remain In Light ALBUM REVIEW
SIMILARITY: 0.883516192

